### Read files

In [33]:
with open('in.txt', 'r') as f:
	data = f.read().splitlines()

with open('test.txt', 'r') as f:
	test = f.read().splitlines()

### Parse and operate functions:
- Parse:
	- Input: list - file read and splitted by lines
	- Output: list of dicts. Each dict represent a monkey. Keys of the dict:
		- id: monkey id [0, n] coincides with the list index, a bit redundant but clearer.
		- items: list with the item values.
		- operation: list with the operator (+, -, *, /, **) and the number to operate the item with
		- test: list - index 0: condition, number to module the item with 0; index 1: monkey ID to throw the item if the condition from idex 0 is met; idex 2: monkey ID if condition is not met.
		- inspected: int - number of time the monkey inspected an item.
	- Observations: None


- Operate:
	- Input: item and list of operations
	- Output: returns the new value of the item. new_item = old_item "operation[0]" "operation[1]"
	- Obs: None

In [34]:
def parse(test):
	monkeys = []
	for line in test:
		if line.strip()[:6] == 'Monkey':
			monkeys.append({'id': int(line.strip()[-2])})
		elif line.strip()[:8] == 'Starting':
			monkeys[-1].update({'items' : [int(x) for x in line.strip()[15:].split(',')]})
		elif line.strip()[:9] == 'Operation':
			if line.strip()[-9:] == 'old * old':
				monkeys[-1].update({'operation': ['**', 2]}) 
			else:
				monkeys[-1].update({'operation': line.strip()[21:].split(' ')})
		elif line.strip()[:4] == 'Test':
			monkeys[-1].update({'test': [int(line.strip()[19:])]})
		elif line.strip()[:7] == 'If true':
			# monkeys[-1].update({'True' : int(line.strip()[-1])})
			monkeys[-1]['test'].append(int(line.strip()[-1]))
		elif line.strip()[:8] == 'If false':
			# monkeys[-1].update({'False' : int(line.strip()[-1])})
			monkeys[-1]['test'].append(int(line.strip()[-1]))
			monkeys[-1].update({'inspected': 0})
		elif line.strip() == '':
			pass
		else:
			print('Error')
			print(line)

	return monkeys

In [35]:
def operate(item, operations):
	if operations[0] == '+':
		item += int(operations[1])
	elif operations[0] == '-':
		item -= int(operations[1])
	elif operations[0] == '*':
		item *= int(operations[1])
	elif operations[0] == '/':
		item /= int(operations[1])
	elif operations[0] == '**':
		item **= int(operations[1])
	return item

### Part 1

In [36]:
def play1(monkeys, rounds=20):
	for round in range(rounds):
		for i in range(len(monkeys)):
			if len(monkeys[i]['items']) != 0:
				# Important to copy the list, otherwise it will be changed during the loop and it messes up the for loop
				items = monkeys[i]['items'].copy()	
				for item in items:
					monkeys[i]['inspected'] += 1
					new_item = operate(item, monkeys[i]['operation'])
					new_item = new_item//3
	
					
					if new_item % monkeys[i]['test'][0] == 0:
						monkeys[monkeys[i]['test'][1]]['items'].append(new_item)
						monkeys[i]['items'].remove(item)
					else:
						monkeys[monkeys[i]['test'][2]]['items'].append(new_item)
						monkeys[i]['items'].remove(item)

	inspected = [monkey['inspected'] for monkey in monkeys]
	biggest = big = 0
	for item in inspected:
		if item > biggest:
			big = biggest
			biggest = item
		elif biggest > item > big:
			big = item
	print("Part 1: ", biggest, big, "Product: ", biggest*big)

### Part 2

The other method is obsolete as the numbers (item's worry values) get very big very fast. To avoid this problem it is very usefull to use the [Chinese Remainder Theorem]

[Chinese Remainder Theorem]: https://brilliant.org/wiki/chinese-remainder-theorem/

In [37]:
def play2(monkeys, rounds=10_000):
	tests = [monkey['test'][0] for monkey in monkeys]
	supermodulus = 1
	for test in tests:
		supermodulus *= test
	for _ in range(rounds):
		for monkey in monkeys:
			if len(monkey['items']) != 0:
				# Important to copy the list, otherwise it will be changed during the loop and it messes up the for loop
				items = monkey['items'].copy()	
				for item in items:
					monkey['inspected'] += 1
					new_item = operate(item, monkey['operation'])
					new_item %= supermodulus
	
					
					if new_item % monkey['test'][0] == 0:
						monkeys[monkey['test'][1]]['items'].append(new_item)
						monkey['items'].remove(item)
					else:
						monkeys[monkey['test'][2]]['items'].append(new_item)
						monkey['items'].remove(item)
						
	inspected = [monkey['inspected'] for monkey in monkeys]
	biggest = big = 0
	for item in inspected:
		if item > biggest:
			big = biggest
			biggest = item
		elif biggest > item > big:
			big = item
	print("Part 2: ", biggest, big, "Product: ", biggest*big)

In [38]:
monkeys = parse(data)
play1(monkeys)
monkeys = parse(data)
play2(monkeys)

Part 1:  572 554 Product:  316888
Part 2:  188167 187442 Product:  35270398814
